In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class ResNet(nn.Module):
    def __init__(self, inchannels, outchannels, kernel_size=3, stride=1, skip=True):
        super().__init__()
        self.skip = skip
        self.block = nn.Sequential(
            nn.Conv2d(inchannels, outchannels, kernel_size=kernel_size, stride=stride, padding=1,bias=False),
            nn.BatchNorm2d(outchannels),
            nn.Dropout(0.1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(outchannels, outchannels, kernel_size=kernel_size, padding=1,bias=False),
            nn.BatchNorm2d(outchannels),
           
        )
        if stride == 2 or inchannels != outchannels:
            self.skip = False
            self.skip_conv = nn.Conv2d(inchannels, outchannels, kernel_size=1, stride=stride,bias=False)
            self.skip_bn = nn.BatchNorm2d(outchannels)
        

    def forward(self, x):
        out = self.block(x)
        if not self.skip:
            out += self.skip_bn(self.skip_conv(x))
        else:
            out += x
        out = F.leaky_relu(out.clone())
        return out


class ResNetF(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7,stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.leakyrelu = nn.LeakyReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        #self.resblock1 = ResNet(32, 32,stride=1)
        #self.resblock2 = ResNet(64, 64,stride=1)
        self.resblock3 = ResNet(64, 64,stride=1)
        #self.resblock4=ResNet(64,64,stride=1)
        #self.resblock5=ResNet(64,64,stride=1)
        self.resblock6=ResNet(64,64,stride=1)
        self.resblock7=ResNet(64,64,stride=1)
        self.resblock8=ResNet(64,128,stride=2)
        self.resblock9=ResNet(128,128,stride=1)
        self.resblock10=ResNet(128,128,stride=1)
        self.resblock11=ResNet(128,128,stride=1)
        self.resblock12=ResNet(128,128,stride=1)
        self.resblock13=ResNet(128,128,stride=1)
        self.resblock14 =ResNet(128,512,stride=2)
       #self.resblock15 =ResNet(512,512,stride=1)
        self.avgpool=nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.flat=nn.Flatten()
        self.fc1= nn.Linear(in_features=512, out_features=10, bias=True)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.leakyrelu(x.clone())
        x = self.maxpool(x)
        #x = self.resblock1(x)
        #x = self.resblock2(x)
        x = self.resblock3(x)
        #x = self.resblock4(x)
        #x = self.resblock5(x)
        x = self.resblock6(x)
        x = self.resblock7(x)
        x = self.resblock8(x)
        x = self.resblock9(x)
        x = self.resblock10(x)
        x = self.resblock11(x)
        x = self.resblock12(x)
        x = self.resblock13(x)
        x= self.resblock14(x)
        #x= self.resblock15(x)
        x = self.avgpool(x)
        x = self.flat(x)
        x = self.fc1(x) 
     
        return x


model = ResNetF()
model=model.cuda()
random_matrix = torch.rand(1, 3, 224, 224).cuda()
print(model.forward(random_matrix).shape)
summary(model,(3,224,224))

torch.Size([1, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
         LeakyReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
           Dropout-7           [-1, 64, 56, 56]               0
         LeakyReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
           ResNet-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
          Dropout-1

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR


def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min = image_min, max = image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image

def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity = 'relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain = nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)

def calculate_accuracy(y_pred, y): # calcualting the accuracy of the model
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc



def train(model, iterator, optimizer, criterion, device): # traing the model on with the images in iterator
    
    epoch_loss = 0
    epoch_acc = 0
   
    model.train()
   
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        optimizer.step()
       
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    lr_scheduler.step()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred= model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [3]:
%%time
transform = transforms.Compose(
    [
     transforms.RandomCrop(size=32, padding=4),
      transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(20),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
     ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True,transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
                            ]))
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
model =ResNetF()

optimizer = optim.SGD(model.parameters(), lr = 1e-1)

criterion = nn.CrossEntropyLoss()
lr_scheduler = CosineAnnealingLR(optimizer, 200)
model = model.to(device)
criterion = criterion.to(device)
  

loss =10
train_loss_history=[]
train_acc_history=[]
test_loss_history=[]
test_acc_history=[]
EPOCHS=200
for i in range(EPOCHS):
    # using train iterator
    train_loss , epoch_acc = train(model,trainloader, optimizer, criterion, device=device)
    print("train loss per epoch => ", i, train_loss, "train acc per epoch=> " , epoch_acc)
    
    epoch_loss , epoch_valid_acc = evaluate(model, testloader, criterion, device)
    print("val loss per epoch => ", epoch_loss , "val acc per epoch =>",epoch_valid_acc,"\n")
        
    train_loss_history.append(train_loss)
    train_acc_history.append(epoch_acc)
    test_loss_history.append(epoch_loss)
    test_acc_history.append(epoch_valid_acc)
    
    if epoch_loss<loss:
        torch.save(model,"./res_512_d.pt")
        loss= epoch_loss


Files already downloaded and verified
Files already downloaded and verified
train loss per epoch =>  0 1.6578762186762621 train acc per epoch=>  0.3909327046340689
val loss per epoch =>  1.4585759654829773 val acc per epoch => 0.48091376582278483 

train loss per epoch =>  1 1.3464124775908488 train acc per epoch=>  0.512775735324606
val loss per epoch =>  1.1972261961502364 val acc per epoch => 0.567246835443038 

train loss per epoch =>  2 1.1993675129797758 train acc per epoch=>  0.5687380116004164
val loss per epoch =>  1.2905958445766303 val acc per epoch => 0.5528085443037974 

train loss per epoch =>  3 1.1019858217910123 train acc per epoch=>  0.6080562659846548
val loss per epoch =>  1.070941504798358 val acc per epoch => 0.633504746835443 

train loss per epoch =>  4 1.0262047707882074 train acc per epoch=>  0.6332201087261404
val loss per epoch =>  1.0602887023853351 val acc per epoch => 0.6332080696202531 

train loss per epoch =>  5 0.971597165707737 train acc per epoch=> 

In [4]:
max(test_acc_history)

0.8666930379746836

In [5]:
np.savetxt("./train_acc_history.txt",train_acc_history)
np.savetxt("./train_loss_history.txt",train_loss_history)
np.savetxt("./test_acc_history.txt",test_acc_history)
np.savetxt("./test_loss_history.txt",test_loss_history)